In [ ]:
import numpy as np
import pandas as pd
import cv2
import matplotlib.pyplot as plt
import glob
import os
from datetime import datetime

# Kaggle input directory
input_dir = '/kaggle/input/visual-quality-inspection'

# List all files in the input directory
print("📁 Arquivos disponíveis no dataset:")
for dirname, _, filenames in os.walk(input_dir):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# Carregar arquivos CSV
train_df = pd.read_csv(os.path.join(input_dir, 'train.csv'))
test_df = pd.read_csv(os.path.join(input_dir, 'test.csv'))
sample_submission = pd.read_csv(os.path.join(input_dir, 'sample_submission.csv'))

print(f"\n📊 Dataset Info:")
print(f"Train samples: {len(train_df)}")
print(f"Test samples: {len(test_df)}")
print(f"Sample submission shape: {sample_submission.shape}")

print(f"\n📋 Sample submission columns:")
print(sample_submission.columns.tolist())

print(f"\n🔍 Train data sample:")
print(train_df.head())

# Configurar pasta de imagens
pasta_imagens = os.path.join(input_dir, 'lego')

# Inicializar o detector SIFT
sift = cv2.SIFT_create()

# Função para encontrar correspondências usando SIFT
def encontrar_correspondencias(img_parte, img_teste, nome_parte):
    gray_parte = cv2.cvtColor(img_parte, cv2.COLOR_RGB2GRAY)
    gray_teste = cv2.cvtColor(img_teste, cv2.COLOR_RGB2GRAY)
    
    kp1, des1 = sift.detectAndCompute(gray_parte, None)
    kp2, des2 = sift.detectAndCompute(gray_teste, None)
    
    if des1 is None or des2 is None:
        return None, None, []
    
    FLANN_INDEX_KDTREE = 1
    index_params = dict(algorithm=FLANN_INDEX_KDTREE, trees=5)
    search_params = dict(checks=50)
    flann = cv2.FlannBasedMatcher(index_params, search_params)
    
    matches = flann.knnMatch(des1, des2, k=2)
    
    good_matches = []
    for match_pair in matches:
        if len(match_pair) == 2:
            m, n = match_pair
            if m.distance < 0.7 * n.distance:
                good_matches.append(m)
    
    return kp1, kp2, good_matches

# Sistema de classificação de defeitos
def classificar_defeitos(resultados):
    limiares = {
        'Chapéu': 10,
        'Face': 1,
        'Cabeça': 3,
        'Braço': 5,
        'Corpo': 10,
        'Mãos': 3,
        'Pernas': 10
    }
    
    mapeamento_defeitos = {
        'Chapéu': 'NO_HAT',
        'Face': 'NO_FACE', 
        'Cabeça': 'NO_HEAD',
        'Braço': 'NO_ARM',
        'Corpo': 'NO_BODY',
        'Mãos': 'NO_HAND',
        'Pernas': 'NO_LEG'
    }
    
    defeitos_encontrados = []
    status_partes = {}
    
    for parte, correspondencias in resultados.items():
        limiar = limiares[parte]
        presente = correspondencias >= limiar
        status_partes[parte] = presente
        
        if not presente:
            defeito = mapeamento_defeitos[parte]
            defeitos_encontrados.append(defeito)
    
    is_compliant = len(defeitos_encontrados) == 0
    
    return {
        'is_compliant': is_compliant,
        'is_defect': not is_compliant,
        'defeitos': defeitos_encontrados,
        'status_partes': status_partes,
        'correspondencias': resultados
    }

# Função para processar uma única imagem
def processar_imagem_completa(caminho_imagem, nome_arquivo):
    try:
        imagem = cv2.imread(caminho_imagem)
        if imagem is None:
            return None
        
        imagem_rgb = cv2.cvtColor(imagem, cv2.COLOR_BGR2RGB)
        altura, largura, _ = imagem.shape
        
        def cortar_partes(img):
            return {
                'Chapéu': img[10:90, 200:297],
                'Face': img[88:100, 220:270],
                'Cabeça': img[88:100, 210:270],
                'Braço': img[90:190, 190:220],
                'Corpo': img[120:170, 215:280],
                'Mãos': img[160:190, 187:220],
                'Pernas': img[170:240, 215:280]
            }
        
        partes = cortar_partes(imagem_rgb)
        
        resultados_sift = {}
        for nome_parte, img_parte in partes.items():
            try:
                kp1, kp2, matches = encontrar_correspondencias(img_parte, imagem_rgb, nome_parte)
                resultados_sift[nome_parte] = len(matches) if matches else 0
            except Exception as e:
                resultados_sift[nome_parte] = 0
        
        resultado_classificacao = classificar_defeitos(resultados_sift)
        
        # Mapear defeitos para colunas do CSV
        defeitos_lista = resultado_classificacao['defeitos']
        
        predicao = {
            'example_id': nome_arquivo.replace('.jpg', ''),
            'has_deffect': 1 if resultado_classificacao['is_defect'] else 0,
            'no_hat': 1 if 'NO_HAT' in defeitos_lista else 0,
            'no_face': 1 if 'NO_FACE' in defeitos_lista else 0,
            'no_head': 1 if 'NO_HEAD' in defeitos_lista else 0,
            'no_leg': 1 if 'NO_LEG' in defeitos_lista else 0,
            'no_body': 1 if 'NO_BODY' in defeitos_lista else 0,
            'no_hand': 1 if 'NO_HAND' in defeitos_lista else 0,
            'no_arm': 1 if 'NO_ARM' in defeitos_lista else 0
        }
        
        return predicao
        
    except Exception as e:
        print(f"Erro ao processar {nome_arquivo}: {e}")
        return None

print(f"\n🚀 Processando imagens do conjunto de teste...")

In [ ]:
# Processar todas as imagens do test.csv e gerar submission
predicoes = []

print("🔍 PROCESSANDO IMAGENS DO TEST SET")
print("="*60)

for i, row in test_df.iterrows():
    example_id = row['example_id']
    
    # Buscar arquivo de imagem correspondente
    imagem_path = None
    for ext in ['jpg', 'jpeg', 'png', 'bmp']:
        caminho_tentativa = os.path.join(pasta_imagens, f"{example_id}.{ext}")
        if os.path.exists(caminho_tentativa):
            imagem_path = caminho_tentativa
            break
    
    if imagem_path is None:
        print(f"[{i+1:3d}/{len(test_df)}] ❌ {example_id} - Imagem não encontrada")
        # Criar predição padrão para imagem não encontrada
        predicao = {
            'example_id': example_id,
            'has_deffect': 1,
            'no_hat': 1,
            'no_face': 1,
            'no_head': 1,
            'no_leg': 1,
            'no_body': 1,
            'no_hand': 1,
            'no_arm': 1
        }
    else:
        print(f"[{i+1:3d}/{len(test_df)}] 🔄 {example_id}")
        
        # Processar imagem
        resultado = processar_imagem_completa(imagem_path, f"{example_id}.jpg")
        
        if resultado is None:
            print(f"    → ❌ Erro no processamento")
            # Criar predição padrão para erro
            predicao = {
                'example_id': example_id,
                'has_deffect': 1,
                'no_hat': 1,
                'no_face': 1,
                'no_head': 1,
                'no_leg': 1,
                'no_body': 1,
                'no_hand': 1,
                'no_arm': 1
            }
        else:
            predicao = resultado
            status = "✅ COMPLIANT" if not resultado['has_deffect'] else "❌ NON-COMPLIANT"
            defeitos_encontrados = [k.replace('no_', '').upper() for k, v in predicao.items() 
                                   if k.startswith('no_') and v == 1]
            defeitos_str = ', '.join(defeitos_encontrados) if defeitos_encontrados else 'NENHUM'
            print(f"    → {status} | Defeitos: {defeitos_str}")
    
    predicoes.append(predicao)

# Criar DataFrame de submission
df_submission = pd.DataFrame(predicoes)

# Garantir que as colunas estejam na ordem correta
colunas_ordenadas = ['example_id', 'has_deffect', 'no_hat', 'no_face', 'no_head', 'no_leg', 'no_body', 'no_hand', 'no_arm']
df_submission = df_submission[colunas_ordenadas]

# Salvar submission
df_submission.to_csv('submission.csv', index=False)

print(f"\n📊 ESTATÍSTICAS FINAIS:")
total_predicoes = len(df_submission)
total_defeitos = df_submission['has_deffect'].sum()
total_compliant = total_predicoes - total_defeitos

print(f"Total de predições: {total_predicoes}")
print(f"COMPLIANT: {total_compliant} ({(total_compliant/total_predicoes)*100:.1f}%)")
print(f"NON-COMPLIANT: {total_defeitos} ({(total_defeitos/total_predicoes)*100:.1f}%)")

print(f"\n🔍 DEFEITOS IDENTIFICADOS:")
defeitos_colunas = ['no_hat', 'no_face', 'no_head', 'no_leg', 'no_body', 'no_hand', 'no_arm']
for col in defeitos_colunas:
    count = df_submission[col].sum()
    nome_defeito = col.replace('no_', '').upper()
    print(f"  - {nome_defeito}: {count} ocorrências ({(count/total_predicoes)*100:.1f}%)")

print(f"\n📋 PRIMEIRAS 10 PREDIÇÕES:")
print(df_submission.head(10))

print(f"\n💾 Arquivo submission.csv criado com sucesso!")
print(f"📁 Shape do submission: {df_submission.shape}")

# Visualização dos resultados
fig, ((ax1, ax2), (ax3, ax4)) = plt.subplots(2, 2, figsize=(15, 10))

# Gráfico 1: Distribuição COMPLIANT vs NON-COMPLIANT
compliant_count = (df_submission['has_deffect'] == 0).sum()
non_compliant_count = (df_submission['has_deffect'] == 1).sum()

labels = ['COMPLIANT', 'NON-COMPLIANT']
sizes = [compliant_count, non_compliant_count]
colors = ['green', 'red']
ax1.pie(sizes, labels=labels, colors=colors, autopct='%1.1f%%', startangle=90)
ax1.set_title('Distribuição das Predições')

# Gráfico 2: Frequência de cada tipo de defeito
defeitos_nomes = [col.replace('no_', '').upper() for col in defeitos_colunas]
defeitos_counts = [df_submission[col].sum() for col in defeitos_colunas]

ax2.bar(defeitos_nomes, defeitos_counts, color='orange')
ax2.set_title('Frequência dos Tipos de Defeito')
ax2.set_ylabel('Número de Ocorrências')
ax2.tick_params(axis='x', rotation=45)

# Gráfico 3: Distribuição do número de defeitos por imagem
df_submission['total_defeitos'] = df_submission[defeitos_colunas].sum(axis=1)
defeitos_dist = df_submission['total_defeitos'].value_counts().sort_index()

ax3.bar(defeitos_dist.index, defeitos_dist.values, color='lightblue')
ax3.set_title('Distribuição do Número de Defeitos por Imagem')
ax3.set_xlabel('Número de Defeitos')
ax3.set_ylabel('Número de Imagens')

# Gráfico 4: Heatmap de correlação entre defeitos
import seaborn as sns
correlacao = df_submission[defeitos_colunas].corr()
sns.heatmap(correlacao, annot=True, cmap='coolwarm', center=0, 
            xticklabels=defeitos_nomes, yticklabels=defeitos_nomes, ax=ax4)
ax4.set_title('Correlação entre Tipos de Defeito')

plt.tight_layout()
plt.show()

print("\n🎉 PROCESSAMENTO COMPLETO!")